In [1]:
alpha_vantage_key = 'YOUR_ALPHA_VANTAGE_KEY'
quandl_key = 'YOUR_QUANDL_KEY'

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import math
from plotly import tools

In [4]:
import plotly.graph_objs as go
from plotly.tools import FigureFactory as FF
import scipy
import peakutils
import numpy as np
import pandas as pd

In [5]:
from alpha_vantage.timeseries import TimeSeries
import matplotlib.pyplot as plt
import plotly as py
from plotly.graph_objs import Scatter, Layout
from datetime import datetime

In [6]:
#I need more data, so gonna use Quandl
import quandl
quandl.ApiConfig.api_key = quandl_key 

In [7]:
init_notebook_mode(connected=True)

In [8]:
def get_stock_data(ticker,start_date,end_date):
    symbol = 'WIKI/' + ticker
    data = quandl.get(symbol,start_date = start_date,end_date = end_date,returns='pandas')
    data.reset_index(inplace=True)
    return data


In [9]:
def find_sharp_points(df,feature='Adj. Close',date='date'):
    cb = np.array(df[feature])
    pos_ind = peakutils.indexes(cb,thres=.02/max(cb),min_dist=.1)
    neg_ind = peakutils.indexes(-cb,thres=.02/max(cb),min_dist=.1)
    peak_dates = []
    valley_dates = []
    for i in pos_ind:
        peak_dates.append(df[date][i])
        
    for j in neg_ind:
        valley_dates.append(df[date][j])
    return (peak_dates,pos_ind,valley_dates,neg_ind)

def bollinger_bands(period,stand_d, data):
    n_avg = data.rolling(window=period).mean()
    n_std = data.rolling(window=period).std()
    up_band = n_avg + (n_std*stand_d)
    low_band = n_avg - (n_std*stand_d)
    return (n_avg,up_band,low_band)


In [10]:
def plot_with_BB(df,period,std,feature = 'Adj. Close',date = 'date'):
    while(df.first_valid_index() != 0): #makes the index starts at zero 
        df.index -= 1
    
    price = go.Scatter(x=df[date],y=df[feature],name='Price')
    
    peak_dates,pos_ind,valley_dates,neg_ind = find_sharp_points(df,feature,date)
    
    peak_points = go.Scatter(x=peak_dates,y=[df[feature][i] for i in pos_ind],name = 'Peak Points',
                   mode = 'markers', marker =dict(size=8,color='rgb(255,0,0)',
                                                 symbol='cross'))

    valley_points = go.Scatter(x=valley_dates,y=[df[feature][i] for i in neg_ind], name= 'Valley Points',
                   mode = 'markers', marker =dict(size=8,color='rgb(0,255,0)',
                                                 symbol='cross'))
    
    
    n_avg,upper_band,lower_band = bollinger_bands(period,std,df[feature])
    ma = go.Scatter(x=df[date],y=n_avg, name ='MA')

    up_band_plot = go.Scatter(x=df[date],y=upper_band, name = 'Upper Band')

    lower_band_plot = go.Scatter(x=df[date],y=lower_band, name = 'Lower Band')
    
    
    iplot([price,peak_points,valley_points,ma,up_band_plot,lower_band_plot])
    

In [11]:
def add_BB_to_data(length, std, df, feature = 'Adj. Close'): #CHANGE IT LATER SO OPTIMIZATION TESTS CAN BE RUN
    length_t = str(length) + '_MA'
    upper_t = 'Upper_Band' #CHANGE THIS
    lower_t = 'Lower_Band' #CHANGE THIS
    n_avg,up_band,low_band = bollinger_bands(length,std,df[feature])
    df[length_t] = n_avg
    df[upper_t] = up_band
    df[lower_t] = low_band

In [12]:
def add_peak_valley_points(df,feature='Adj. Close',date='Date'):
    peak_dates,pos_ind,valley_dates,neg_ind = find_sharp_points(df,feature = feature,date = date)
    peak=[]
    k = 0
    while(k < len(df)):
        if(k in pos_ind):
            peak.append(1)
        else:
            peak.append(0)
    
        k = k+1
    
    valleys=[]
    z = 0
    while(z< len(df)):
        if(z in neg_ind):
            valleys.append(1)
        else:
            valleys.append(0)
        z = z+1
    
    df['peak_point'] = peak
    df['valley_point'] = valleys
    
    

In [13]:
def find_signals(df):
    temp_price_peak = -1
    temp_price_valley = -1
    i = 1
    df['Signal'] = 0
    while(i<len(df)):
        curr_price = df['Adj. Close'][i]
        curr_date = df['Date'][i]
        if(df['peak_point'][i-1] == 1):
            temp_price_peak = df['Adj. Close'][i-1]
            temp_peak_ind = i-1
            
        if(df['valley_point'][i-1] == 1):
            temp_price_valley = df['Adj. Close'][i-1]
            temp_valley_ind = i-1
            
        if(curr_price > df['Upper_Band'][i]):
            if(temp_price_peak != -1 and curr_price > temp_price_peak and temp_price_valley != -1 and temp_price_valley > df['Upper_Band'][temp_valley_ind]):
                df['Signal'][i] = ('Long',curr_date,curr_price,i)
    
        if(curr_price < df['Lower_Band'][i]):
            if(temp_price_valley != -1 and curr_price < temp_price_valley and temp_price_peak != -1 and temp_price_peak < df['Lower_Band'][temp_peak_ind]):
                df['Signal'][i] = ('Short',curr_date,curr_price,i)
        i = i+ 1

In [14]:
def clean_signals_list(df):
    sig_list = []
    i = 1
    while(i < len(df)-1):
        k = df['Signal'][i]
        if df['Signal'][i-1] == 0:
            sig_list.append(k)
        i = i + 1
    
    sig_list = list(filter(lambda x: x!=0, sig_list)) #remove zeroes
    return sig_list

In [15]:
def long_and_short_info(signals_list):
    buy_date = []
    buy_price = []
    buy_index = []
    short_date = []
    short_price = []
    short_index = []
    for i in signals_list:
        if i[0] == 'Long':
            buy_date.append(i[1])
            buy_price.append(i[2])
            buy_index.append(i[3])
        if i[0] == 'Short':
            short_date.append(i[1])
            short_price.append(i[2])
            short_index.append(i[3])
            
    return(buy_date,buy_price,buy_index,short_date,short_price,short_index)



In [16]:
def calculate_postion_list(buy_date,buy_price,buy_index,short_date,short_price,short_index,data):
    has_postion = True
    postion_list = []
    i = 0
    while(i< len(buy_price)):
        buy_price_i = buy_price[i]
        buy_date_i = buy_date[i]
        buy_index_i = buy_index[i]
        temp_ind = buy_index_i
        if(temp_ind <= buy_index_i):
            has_postion = True
            while(has_postion and temp_ind < len(data)):
                curr_price = data['Adj. Close'][temp_ind]
                profit = (curr_price-buy_price_i)/buy_price_i
                if profit >= .02:
                    postion_list.append((data['Date'][buy_index_i],data['Date'][temp_ind],'Long',buy_price_i,curr_price,profit))
                    has_postion = False
                if profit <= -.01:
                    postion_list.append((data['Date'][buy_index_i],data['Date'][temp_ind],'Long',buy_price_i,curr_price,profit))
                    has_postion = False
                temp_ind = temp_ind + 1
        i = i +1
    #-------------------------------------------------------------------------
    has_postion = True
    i = 0
    while(i< len(short_price)):
        short_price_i = short_price[i]
        short_date_i = short_date[i]
        short_index_i = short_index[i]
        temp_ind = short_index_i
        if(temp_ind <= short_index_i):
            has_postion = True
            while(has_postion and temp_ind < len(data)):
                curr_price = data['Adj. Close'][temp_ind]
                profit = (curr_price-short_price_i)/short_price_i
                if profit <= -.02:
                    neg_profit = profit*-1
                    postion_list.append((data['Date'][short_index_i],data['Date'][temp_ind],'Short',short_price_i,curr_price,neg_profit))
                    has_postion = False
                if profit >= .01:
                    postion_list.append((data['Date'][short_index_i],data['Date'][temp_ind],'Short',short_price_i,curr_price,profit))
                    has_postion = False
                temp_ind = temp_ind + 1
        i = i +1
    #------------------------------------------------------------------------------
    i = 0
    while(i < len(postion_list)):
        if(i + 1 == len(postion_list)):
            if(postion_list[i][1] >= postion_list[-1][0]):
                del postion_list[-1]
                break
        if(postion_list[i][1] >= postion_list[i+1][0]):
            del postion_list[i+1]
        i = i + 1
    #-------------------------------------------------------------------------
    return sorted(postion_list)

In [17]:
def plot_strat_points(postion_list,data):
    date_long_pos_taken = []
    price_long_pos_taken = []
    date_long_pos_ended = []
    price_long_pos_ended = []

    date_short_pos_taken = []
    price_short_pos_taken = []
    date_short_pos_ended = []
    price_short_pos_ended = []

    for i in postion_list:
        if(i[2] == 'Long'):
            date_long_pos_taken.append(i[0])
            date_long_pos_ended.append(i[1])
            price_long_pos_taken.append(i[3])
            price_long_pos_ended.append(i[4])
        else:
            date_short_pos_taken.append(i[0])
            date_short_pos_ended.append(i[1])
            price_short_pos_taken.append(i[3])
            price_short_pos_ended.append(i[4])
    
    price = go.Scatter(x=data['Date'], y=data['Adj. Close'], name = 'Price')
    long_buy_points = go.Scatter(x=date_long_pos_taken,y=price_long_pos_taken, name = 'Long Buy Points',mode = 'markers',marker = dict(size=8,color='green',
                                                                                                 symbol='o'))
    long_sell_points = go.Scatter(x=date_long_pos_ended,y=price_long_pos_ended, name = 'Long Sell Points',mode = 'markers',marker = dict(size=8,color='red',
                                                                                                 symbol='o'))
    short_sell_points = go.Scatter(x=date_short_pos_taken,y=price_short_pos_taken, name = 'Short Sell Points',mode = 'markers',marker = dict(size=8,color='rgb(224,224,0)',
                                                                                                 symbol='o'))
    short_buy_points = go.Scatter(x=date_short_pos_ended,y=price_short_pos_ended, name = 'Short Buy Points',mode = 'markers',marker = dict(size=8,color='rgb(0,0,0)',
                                                                                                 symbol='o'))
    
    
    iplot([price,long_buy_points,long_sell_points,short_sell_points,short_buy_points])
    

In [18]:
def plot_strat_with_BB(postion_list,data):
        date_long_pos_taken = []
        price_long_pos_taken = []
        date_long_pos_ended = []
        price_long_pos_ended = []

        date_short_pos_taken = []
        price_short_pos_taken = []
        date_short_pos_ended = []
        price_short_pos_ended = []

        for i in postion_list:
            if(i[2] == 'Long'):
                date_long_pos_taken.append(i[0])
                date_long_pos_ended.append(i[1])
                price_long_pos_taken.append(i[3])
                price_long_pos_ended.append(i[4])
            else:
                date_short_pos_taken.append(i[0])
                date_short_pos_ended.append(i[1])
                price_short_pos_taken.append(i[3])
                price_short_pos_ended.append(i[4])
    
        price = go.Scatter(x=data['Date'], y=data['Adj. Close'], name = 'Price')
        long_buy_points = go.Scatter(x=date_long_pos_taken,y=price_long_pos_taken, name = 'Long Buy Points',mode = 'markers',marker = dict(size=8,color='green',
                                                                                                 symbol='o'))
        long_sell_points = go.Scatter(x=date_long_pos_ended,y=price_long_pos_ended, name = 'Long Sell Points',mode = 'markers',marker = dict(size=8,color='red',
                                                                                                 symbol='o'))
        short_sell_points = go.Scatter(x=date_short_pos_taken,y=price_short_pos_taken, name = 'Short Sell Points',mode = 'markers',marker = dict(size=8,color='rgb(224,224,0)',
                                                                                                 symbol='o'))
        short_buy_points = go.Scatter(x=date_short_pos_ended,y=price_short_pos_ended, name = 'Short Buy Points',mode = 'markers',marker = dict(size=8,color='rgb(0,0,0)',
                                                                                                symbol='o'))
        while(data.first_valid_index() != 0): #makes the index starts at zero 
            data.index -= 1
    
        peak_dates,pos_ind,valley_dates,neg_ind = find_sharp_points(data,feature='Adj. Close',date='Date')
    
        peak_points = go.Scatter(x=peak_dates,y=[data['Adj. Close'][i] for i in pos_ind],name = 'Peak Points',
                   mode = 'markers', marker =dict(size=8,color='rgb(255,0,0)',
                                                 symbol='cross'))

        valley_points = go.Scatter(x=valley_dates,y=[data['Adj. Close'][i] for i in neg_ind], name= 'Valley Points',
                   mode = 'markers', marker =dict(size=8,color='rgb(0,255,0)',
                                                 symbol='cross'))
    
    
        n_avg,upper_band,lower_band = bollinger_bands(21,.312,data['Adj. Close'])
        ma = go.Scatter(x=data['Date'],y=n_avg, name ='MA')

        up_band_plot = go.Scatter(x=data['Date'],y=upper_band, name = 'Upper Band')

        lower_band_plot = go.Scatter(x=data['Date'],y=lower_band, name = 'Lower Band')
        iplot([price,long_buy_points,long_sell_points,short_buy_points,short_sell_points,up_band_plot,ma,lower_band_plot,peak_points,valley_points])
    

In [19]:
def make_index_0(df):
    while(df.first_valid_index() != 0): #makes the index starts at zero 
        df.index -= 1

In [20]:
def apply_strat(df):
    while(df.first_valid_index() != 0): #makes the index starts at zero 
        df.index -= 1
    
    
    find_signals(df) #populate signals
    sig_list = clean_signals_list(df) #get clean signals list
    buy_date,buy_price,buy_index,short_date,short_price,short_index = long_and_short_info(sig_list)
    postion_list = calculate_postion_list(buy_date,buy_price,buy_index,short_date,short_price,short_index,data = df)
    plot_strat_points(postion_list,df)
    return(postion_list)

In [21]:
def truncate_num(num):
    """This chops truncates the number so that it returns the first 4 digits of the number"""
    num = num *10000
    num = int(num)
    return num/10000

In [22]:
def wins_losses_precents(postion_list):
    wins = []
    losses = []
    precents = []
    for i in postion_list:
        precents.append((i[1],i[5]))
        if (i[5] > 0):
            wins.append((i[1],i[5]))
        else:
            losses.append((i[1],i[5]))
    return (wins,losses,precents)

In [23]:
def plot_win_loss_bar(wins,losses):
    win_plot = go.Bar(x=[i[0] for i in wins], y = [i[1] for i in wins], name = 'Win', marker=dict(
        color='rgb(50,205,50)', 
        line=dict(
            color='rgb(0,100,0)',
            width=1.5,)))
    loss_plot = go.Bar(x=[i[0] for i in losses], y = [i[1] for i in losses], name = 'Loss',marker=dict(
        color='rgb(255,0,0)', 
        line=dict(
            color='rgb(139,0,0)',
            width = 1.5,)))
    iplot([win_plot,loss_plot])
    

In [24]:
def plot_yeild(capital_list, precent_list):
    yeild_curve = go.Scatter(x = sorted([i[0] for i in capital_list]), y = [i[1] for i in capital_list], name="Capital",line = dict(color = '#ff8f40'))

    precent_plot = go.Bar(x=[i[0] for i in precent_list], y = [i[1] for i in precent_list], name = 'Percent Changes', marker=dict(
            color='#4d61c1', 
            line=dict(
                color='#4d61c1',
                width=1.5,
            )
        ))

    fig = tools.make_subplots(rows=2, cols=1,print_grid=False)

    fig.append_trace(yeild_curve, 1, 1)
    fig.append_trace(precent_plot, 2, 1)

    fig['layout'].update(height=500, width=1000, title='Portfolio Overview')
    iplot(fig)

In [25]:
def performance(capital_list,percent_list,postion_list,data,feature='Adj. Close'):
    if(len(capital_list) == 0):
        return [957,957,957,957,957,957,957,957,957,957,957,957,957,957,957]
    
    num_wins = 0
    num_losses = 0
    for i in percent_list:
        if(i[1]>0):
            num_wins = num_wins + 1
        else:
            num_losses = num_losses + 1
            
    if(num_wins != 0):
        win_rate = float('%.2f' % (num_wins / (num_wins + num_losses) * 100))
    else:
        win_rate = float(0)
    
    
    int_cap = float("%.2f" % (capital_list[0][1]/(1+percent_list[0][1])))
    final_cap = float("%.2f" % capital_list[-1][1])
    num_of_trades = len(percent_list)
    int_date = str(percent_list[0][0])
    int_date = int_date.split(' ')[0]
    fin_date = str(percent_list[-1][0])
    fin_date = fin_date.split(' ')[0]
    num_days = percent_list[-1][0] - percent_list[0][0]
    num_years = float("%.2f" % (num_days.total_seconds()/31556926))
    num_days = float('%.2f' % (num_days.total_seconds()/86400))

    
    length_of_trades = []
    for i in postion_list:
        diff = i[1] - i[0]
        diff = '%.2f' % (diff.total_seconds()/86400)
        length_of_trades.append(float(diff))
    
    avg_length = float('%.2f' % (sum(length_of_trades)/len(length_of_trades)))
    max_length = max(length_of_trades)
    min_length = min(length_of_trades)
    buy_hold_return = float('%.2f' % ((data[feature][-1:] - data[feature][0])/data[feature][0] * 100))
    strat_return = float('%.2f' % ((capital_list[-1][1] - int_cap)/int_cap * 100))
    #-------------------------------------------------------------------------------------------------------------------------
    
    
    print('First Date Traded: ' + str(int_date))
    print('Last Date Traded: ' + str(fin_date))
    print('Total Duration(in Days): ' + str(num_days))
    print('Total Duration(in Years): ' + str(num_years))
    print('Avg. Duration of Trade(in Days): ' + str(avg_length))
    print('Max Duration of Trade(in Days): ' + str(max_length))
    print('Min Duration of Trade(in Days): ' + str(min_length))
    print('Buy and Hold Returns: ' + str(buy_hold_return) + '%')
    print('Strategy Returns: ' + str(strat_return) + '%')
    print('Total Number of Trades: ' + str(num_of_trades))
    print('Number of Wins: ' + str(num_wins))
    print('Number of Losses: ' + str(num_losses))
    print('Win Rate: ' + str(win_rate) + '%')
    print('Intial Capital: ' + str(int_cap))
    print('Final Capital: ' + str(final_cap))
    
    #--------------------------------------------------------------------------------------------------------------------------
    return[int_date,fin_date,num_of_trades, num_days,num_years,avg_length,max_length,min_length,buy_hold_return, strat_return,num_wins,num_losses,win_rate,int_cap,final_cap]


In [26]:
#Lets test it out on some stocks

In [27]:
def do_it_all(data1,intial_capital): 
    data = data1 #make a copy of the data just in case 
    add_BB_to_data(21,.312,data,feature='Adj. Close')
    add_peak_valley_points(data,feature='Adj. Close',date='Date')
    p_l = apply_strat(data)
    wins,losses,precents = wins_losses_precents(p_l)
    cap_list = get_cap_list(p_l,intial_capital)
    metrics = performance(cap_list,precents,p_l,data,feature = 'Adj. Close')
    return (wins,losses,cap_list,precents,metrics)

In [28]:
def get_cap_list(postion_list,intial_capital = 100):
    cap_list = [(0,intial_capital)]
    i = 0
    while(i < len(postion_list)):
        profit = cap_list[i][1]*(1+postion_list[i][5])
        cap_list.append((postion_list[i][1],profit))
        i = i + 1
    
    del cap_list[0]
    return cap_list

In [29]:
def get_bhs_cap_list(data,feature='Adj. Close',date='Date',intial_capital=100):
    buy_hold_info = []
    i = 0
    while(i < len(data)):
        if( (i + 1) == len(data)):
            precent_change = (data[feature][-1:] - data[feature][i])/(data[feature][i])
            date1 = data[date][i]
            buy_hold_info.append((date1,precent_change))
        else:
            precent_change = (data[feature][i+1] - data[feature][i])/(data[feature][i])
            date1 = data[date][i]
            buy_hold_info.append((date1,precent_change))
        
        i = i + 1

    del buy_hold_info[0]
    cap_list = [(0,intial_capital)]
    i = 0
    while(i< len(buy_hold_info)):
        num = truncate_num(buy_hold_info[i][1])
        if(num >=0):
            port_value = cap_list[i][1]*(1+ num)
            cap_list.append((buy_hold_info[i][0],port_value))
        else:
            port_value = cap_list[i][1]*(1- abs(num))
            cap_list.append((buy_hold_info[i][0],port_value))
    
        i = i + 1
    del cap_list[0]
    return cap_list
    

In [30]:
def compare_bhs_to_strat(bhs_cap_list,strat_cap_list,snp_cap_list):
    strat_curve = go.Scatter(x = sorted([i[0] for i in strat_cap_list]), y = [i[1] for i in strat_cap_list], name="Strat",line = dict(color = '#ff8f40'))
    bhs_curve = go.Scatter(x = sorted([i[0] for i in bhs_cap_list]), y = [i[1] for i in bhs_cap_list], name="BHS",line = dict(color = '#ABCDEF'))
    snp_curve = go.Scatter(x = sorted([i[0] for i in snp_cap_list]), y = [i[1] for i in snp_cap_list], name="S&P 500",line = dict(color = '#0d6300'))
   
    iplot([strat_curve,bhs_curve,snp_curve])

In [31]:
def get_snp_cap_list(start_date,end_date,i_cap):
    snp = quandl.get('YALE/SPCOMP' ,start_date=start_date,end_date=end_date)
    snp.reset_index(inplace=True)
    snp['Close'] = snp['Real Price']
    snp['Date'] = snp['Year'] 
    snp_cap_list = get_bhs_cap_list(snp,intial_capital = i_cap,feature = 'Close')
    
    return snp_cap_list


In [32]:
metrics_list = ['First Date Traded','Last Date Traded','Number of Trades','Number of Days','Number of Years','Avg. Length of Trade','Max Length of Trade','Min Length of Trade','BHS','Strat','Number of Wins','Number of Losses','Win Rate','Inital Cap.','Final Cap.']

In [33]:
def get_stock_metrics(stock,intial_capital):
    add_BB_to_data(21,.312,stock)
    add_peak_valley_points(stock)
    p_l_stock = apply_strat(stock)
    wins_stock,losses_stock,precents_stock = wins_losses_precents(p_l_stock)
    stock_cap_strat = get_cap_list(p_l_stock,intial_capital)
    stock_bhs_strat = get_bhs_cap_list(stock)
    stock_metrics = performance(stock_cap_strat,precents_stock,p_l_stock,stock)
    return stock_metrics

In [34]:
s_date='1999-01-01'
e_date='2018-04-25'

In [36]:
def get_stock_data_av(ticker,output):
    ts = TimeSeries(key=alpha_vantage_key, output_format='pandas')
    data, meta_data = ts.get_daily_adjusted(symbol=ticker, outputsize=output)
    data.reset_index(inplace=True)
    data['Adj. Close'] = data['5. adjusted close'] 
    data['Date'] = data['date']
    dates = []
    for i in data.index:
        dates.append(pd.to_datetime(data['Date'][i],yearfirst=True))
    
    data['Date'] = dates
    return data

In [37]:
def log_allocation(num_assets,intial_capital):
    ci = intial_capital
    alloca = []
    i = 0
    k = 0
    x = num_assets
    while(i<x):
        k += pow(1/2,i)
        i += 1

    y = pow(k,-1)
    i = 0
    while(i<x):
        alloca.append(ci*pow(y,i+1))
        i +=1
    alloca_list = []
    for i in alloca:
        alloca_list.append(truncate_num(i))
    return alloca_list

In [38]:
def log_allocation_returns(df):
    temp = df.sort_values(by=['Strat'],ascending=False)
    alloca_list = log_allocation(len(temp),10000)
    i = 0
    returns_list = []
    while(i<len(temp)):
        ret_i = (((temp['Strat'][i])/100)+1)*alloca_list[i]
        returns_list.append(ret_i)
        i += 1
    print(str(truncate_num((sum(returns_list)/10000)-1)*100)+'%')

In [39]:
def get_intraday_av(ticker,timeFrame,size):
    ts = TimeSeries(key=alpha_vantage_key, output_format='pandas')
    data,meta_data = ts.get_intraday(ticker,timeFrame,size)
    data.reset_index(inplace=True)
    data['Adj. Close'] = data['4. close'] 
    data['Date'] = data['date']
    dates = []
    for i in data.index:
        dates.append(pd.to_datetime(data['Date'][i],yearfirst=True))
    
    data['Date'] = dates
    return data

In [40]:
def add_returns(df):
    i = 1
    ret_list = []
    while(i<len(df)):
        returns = (df['Adj. Close'][i]-df['Adj. Close'][i-1])/df['Adj. Close'][i-1]
        ret_list.append(truncate_num(returns))
        i += 1
    ret_list.append(0)
    df['Returns'] = ret_list

In [41]:
def calculate_postion_list(df):
    i = 0
    has_postion = False
    postion_list = []
    while(i<len(df)):
        if(type(df['Signal'][i]) == tuple):
            if(df['Signal'][i][0] == 'Long'):
                price_taken = df['Signal'][i][2]
                index_taken = df['Signal'][i][3]
                temp_price = df['Adj. Close'][i]
                k = index_taken
                while(sum(df['Returns'][index_taken:k]) < .02 and sum(df['Returns'][index_taken:k]) >-.01 and k<len(df)-1):
                     k = k+1
                date_exit = df['Date'][k]
                postion_list.append((df['Signal'][index_taken][1], date_exit,'Long',price_taken,df['Adj. Close'][k],sum(df['Returns'][index_taken:k])))
                
            if(df['Signal'][i][0] == 'Short'):
                price_taken = df['Signal'][i][2]
                index_taken = df['Signal'][i][3]
                temp_price = df['Adj. Close'][i]
                k = index_taken
                while(sum(df['Returns'][index_taken:k]) > -.02 and sum(df['Returns'][index_taken:k]) <.01 and k<len(df)-1):
                    k = k+1
                date_exit = df['Date'][k]
                postion_list.append((df['Signal'][index_taken][1], date_exit,'Short',price_taken,df['Adj. Close'][k],-sum(df['Returns'][index_taken:k])))
        i += 1
    #-----------------------------------------------------------------------------------------------------------------------------------------------
    #Every postion possible got calculated, now to remove double postions
    #------------------------------------------------------------------------------------------------
    i = 1
    bad_list = []
    while(i<len(postion_list)):
        if(postion_list[i-1][1] >= postion_list[i][0]):
            bad_list.append(postion_list[i])
        i = i + 1
    
    good_list = [x for x in postion_list if x not in bad_list]
    #----------------------------------------
    return good_list
    

In [50]:
aal = get_stock_data_av('AAL','full')
add_BB_to_data(21,.3,aal)
add_peak_valley_points(aal)
add_returns(aal)
find_signals(aal)
p_l_aal = calculate_postion_list(aal)
wins,losses,per = wins_losses_precents(p_l_aal)
cap_list = get_cap_list(p_l_aal)
plot_yeild(cap_list,per)
stock_stats = performance(cap_list,per,p_l_aal,aal)

First Date Traded: 2005-10-28
Last Date Traded: 2018-05-10
Total Duration(in Days): 4577.0
Total Duration(in Years): 12.53
Avg. Duration of Trade(in Days): 2.69
Max Duration of Trade(in Days): 15.0
Min Duration of Trade(in Days): 1.0
Buy and Hold Returns: 134.41%
Strategy Returns: 428.59%
Total Number of Trades: 280
Number of Wins: 133
Number of Losses: 147
Win Rate: 47.5%
Intial Capital: 100.0
Final Capital: 528.59


In [51]:
plot_win_loss_bar(wins,losses)

In [52]:
plot_strat_points(p_l_aal,aal)

In [43]:
def stock_figures(ticker):
    stock = get_stock_data_av(ticker,'full')
    add_BB_to_data(21,.3,stock)
    add_peak_valley_points(stock)
    add_returns(stock)
    find_signals(stock)
    p_l_stock = calculate_postion_list(stock)
    wins,losses,per = wins_losses_precents(p_l_stock)
    cap_list = get_cap_list(p_l_stock)
    plot_yeild(cap_list,per)
    stock_stats = performance(cap_list,per,p_l_stock,stock)
    return stock_stats

In [35]:
#Market Cap.: >= 10 Bln
#Avg. Volume: > 1M
#Relative Volume: >.75 (ratio between current volume and 3 month avg.)
#Country: USA
#Option/Short: Both
#Price: <$50
universe = ['AAL','BLL','CHD','CNP','DISCA','DISH','FDC','FOXA','HOLX','HST','KKR','MDLZ','MET','MRO','NLY','NRG','NWL','S','SIRI','SNAP','VIAB','VST','XRAY']

In [46]:
stock_metrics = pd.DataFrame(index=metrics_list)

In [47]:
for i in universe:
    stock_stats = stock_figures(i)
    stock_metrics[i]= stock_stats

First Date Traded: 2005-10-28
Last Date Traded: 2018-05-10
Total Duration(in Days): 4577.0
Total Duration(in Years): 12.53
Avg. Duration of Trade(in Days): 2.69
Max Duration of Trade(in Days): 15.0
Min Duration of Trade(in Days): 1.0
Buy and Hold Returns: 134.41%
Strategy Returns: 428.59%
Total Number of Trades: 280
Number of Wins: 133
Number of Losses: 147
Win Rate: 47.5%
Intial Capital: 100.0
Final Capital: 528.59


First Date Traded: 2000-02-07
Last Date Traded: 2018-05-16
Total Duration(in Days): 6673.0
Total Duration(in Years): 18.27
Avg. Duration of Trade(in Days): 5.17
Max Duration of Trade(in Days): 32.0
Min Duration of Trade(in Days): 1.0
Buy and Hold Returns: 1734.58%
Strategy Returns: -36.81%
Total Number of Trades: 333
Number of Wins: 134
Number of Losses: 199
Win Rate: 40.24%
Intial Capital: 100.0
Final Capital: 63.19


First Date Traded: 2000-02-07
Last Date Traded: 2018-05-03
Total Duration(in Days): 6660.0
Total Duration(in Years): 18.23
Avg. Duration of Trade(in Days): 6.8
Max Duration of Trade(in Days): 63.0
Min Duration of Trade(in Days): 1.0
Buy and Hold Returns: 1257.96%
Strategy Returns: -13.77%
Total Number of Trades: 284
Number of Wins: 120
Number of Losses: 164
Win Rate: 42.25%
Intial Capital: 100.0
Final Capital: 86.23


First Date Traded: 2000-02-28
Last Date Traded: 2018-05-23
Total Duration(in Days): 6659.0
Total Duration(in Years): 18.23
Avg. Duration of Trade(in Days): 5.0
Max Duration of Trade(in Days): 30.0
Min Duration of Trade(in Days): 1.0
Buy and Hold Returns: 177.32%
Strategy Returns: 0.46%
Total Number of Trades: 334
Number of Wins: 127
Number of Losses: 207
Win Rate: 38.02%
Intial Capital: 100.0
Final Capital: 100.46


First Date Traded: 2005-08-23
Last Date Traded: 2018-05-24
Total Duration(in Days): 4657.0
Total Duration(in Years): 12.75
Avg. Duration of Trade(in Days): 3.88
Max Duration of Trade(in Days): 21.0
Min Duration of Trade(in Days): 0.0
Buy and Hold Returns: 222.04%
Strategy Returns: -54.64%
Total Number of Trades: 256
Number of Wins: 88
Number of Losses: 168
Win Rate: 34.38%
Intial Capital: 100.0
Final Capital: 45.36


...

First Date Traded: 2000-02-17
Last Date Traded: 2018-05-16
Total Duration(in Days): 6663.0
Total Duration(in Years): 18.24
Avg. Duration of Trade(in Days): 3.05
Max Duration of Trade(in Days): 28.0
Min Duration of Trade(in Days): 1.0
Buy and Hold Returns: -82.65%
Strategy Returns: 111.36%
Total Number of Trades: 358
Number of Wins: 160
Number of Losses: 198
Win Rate: 44.69%
Intial Capital: 100.0
Final Capital: 211.36


First Date Traded: 2017-04-18
Last Date Traded: 2018-05-24
Total Duration(in Days): 401.0
Total Duration(in Years): 1.1
Avg. Duration of Trade(in Days): 2.53
Max Duration of Trade(in Days): 9.0
Min Duration of Trade(in Days): 1.0
Buy and Hold Returns: -56.7%
Strategy Returns: 63.32%
Total Number of Trades: 19
Number of Wins: 11
Number of Losses: 8
Win Rate: 57.89%
Intial Capital: 100.0
Final Capital: 163.32


First Date Traded: 2006-01-31
Last Date Traded: 2018-05-24
Total Duration(in Days): 4496.0
Total Duration(in Years): 12.31
Avg. Duration of Trade(in Days): 3.96
Max Duration of Trade(in Days): 33.0
Min Duration of Trade(in Days): 1.0
Buy and Hold Returns: -5.52%
Strategy Returns: -53.78%
Total Number of Trades: 223
Number of Wins: 87
Number of Losses: 136
Win Rate: 39.01%
Intial Capital: 100.0
Final Capital: 46.22


First Date Traded: 2016-11-28
Last Date Traded: 2018-05-02
Total Duration(in Days): 520.0
Total Duration(in Years): 1.42
Avg. Duration of Trade(in Days): 5.08
Max Duration of Trade(in Days): 13.0
Min Duration of Trade(in Days): 1.0
Buy and Hold Returns: 75.11%
Strategy Returns: 31.92%
Total Number of Trades: 25
Number of Wins: 15
Number of Losses: 10
Win Rate: 60.0%
Intial Capital: 100.0
Final Capital: 131.92


First Date Traded: 2000-03-23
Last Date Traded: 2018-05-24
Total Duration(in Days): 6636.0
Total Duration(in Years): 18.17
Avg. Duration of Trade(in Days): 4.65
Max Duration of Trade(in Days): 29.0
Min Duration of Trade(in Days): 1.0
Buy and Hold Returns: 552.31%
Strategy Returns: -71.12%
Total Number of Trades: 330
Number of Wins: 110
Number of Losses: 220
Win Rate: 33.33%
Intial Capital: 100.0
Final Capital: 28.88


In [48]:
sm_f = stock_metrics.T

In [49]:
strat = go.Bar(y=stock_metrics.loc['Strat'],name='Strat')
bhs = go.Bar(y=stock_metrics.loc['BHS'],name='BHS')
iplot([strat,bhs])